<a href="https://colab.research.google.com/github/aleks-haksly/ClickHouse/blob/main/preparation_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Практическая работа по оптимизации занимаемого на диске места таблицы ClickHouse

### Настраиваем подключение

In [ ]:
!pip install clickhouse_driver -q

In [ ]:
# Импортируем используемые в работе модули
import pandas as pd
from clickhouse_driver import Client
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import userdata

In [ ]:
# Реквизиты для подключения к БД
user = 'hardda_student_alekse-fomin'
password = userdata.get('HA[hardda_student_alekse-fomin]')

host='clickhouse.lab.karpov.courses'
port=9000
database_r='hardda' # БД с правами только на чтение
database_rw='hardda_student_data' # БД с правами чтение и запись

source_table = "live_adverts_dwh" # Таблица до оптимизации
optimized_table = "dp_live_adverts_alekse_fomin" # Таблица после оптимизации

In [ ]:
# Создаем соединение с БД ClickHouse

# Соединение
client_r = Client(host=host, port=port, user=user,
                password=password, database=database_r)


### Соберем предварительную информацию об оптимизируемой таблице

In [ ]:
# Смотрим на таблицу
sql = f"""
SELECT
  *
FROM
  {source_table}
LIMIT 5
"""
result, columns = client_r.execute(sql, with_column_types=True)
df = pd.DataFrame(result, columns=[tuple[0] for tuple in columns])
df.head(5)

,execution_date,advert_id,created_at,price,region,user_id,platform,auto_brand,auto_model,passport_id,year,userType
0,2022-02-01,130486371,2012-11-06 07:12:00,0,Москва,123578197,unknown,Unknown,Unknown model,123599132,0,0
1,2022-02-01,129236743,2012-11-06 07:31:13,0,Москва,123477706,unknown,Unknown,Unknown model,123493336,0,0
2,2022-02-01,130358537,2012-11-06 07:35:24,0,Москва,123799514,unknown,Unknown,Unknown model,123820472,0,0
3,2022-02-01,129230893,2012-11-06 08:01:17,0,Москва,123730252,unknown,Unknown,Unknown model,123751205,0,0
4,2022-02-01,169982253,2012-11-06 08:24:59,0,Москва,123653651,unknown,Unknown,Unknown model,123674596,0,0


[(datetime.date(2022, 2, 1),
  datetime.date(2022, 1, 31),
  'android',
  'f7411212fd0e2523e126cbfdd3f226c211212',
  '4beb10e1-aeeb-4c52-acd2-ce1ddbc1fc24b10e1',
  22,
  11,
  3,
  0,
  0,
  0,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  0),
 (datetime.date(2022, 2, 1),
  datetime.date(2022, 1, 31),
  'android',
  'b58f68c4ff9044b05a69a6cf2e745dd3f68c4',
  'f7b420a7-0fd0-48cd-b218-a804b24427ee420a7',
  79,
  41,
  7,
  0,
  0,
  0,
  5,
  0,
  10,
  0,
  1,
  0,
  0,
  0),
 (datetime.date(2022, 2, 1),
  datetime.date(2022, 1, 31),
  'android',
  '99cb5b2be630ac498503e9671627c92bb5b2b',
  '78509856-1dec-45c6-a308-646a10e7c38a09856',
  3,
  1,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0),
 (datetime.date(2022, 2, 1),
  datetime.date(2022, 1, 31),
  'android',
  '71071335efe3a0a0504b4cec67e6b26571335',
  '1d67c699-432d-4368-8ef8-d67f17ad55b97c699',
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0),
 (datetime.date(2022, 2, 1),
  datetime.date(2022, 1, 31),
  'a